In [1]:
# drive'a bağlanarak başlayalım.

from google.colab import drive
drive.mount('\content\drive')

KeyboardInterrupt: 

# Online Ödeme Dolandırıcılığı Tespiti

## Denetimli Öğrenme

### Giriş
İnternet üzerinden ödeme, günümüzde en popüler işlem yöntemlerinden biridir. Ancak, çevrimiçi ödemelerdeki artışla birlikte ödeme dolandırıcılııklarında da bir yükseliş gözlemlenmektedir.
Bu proje, denetimli ve denetimsiz öğrenme tekniklerini kullanarak çevrimiçi ödeme işlemlerinde dolandırıcılığı tespit etmeyi amaçlamaktadır. Dolandırıcılığı doğru ve verimli bir şekilde tespit etmek için çeşitli makine öğrenmesi modelleri test edilmiş ve değerlendirilmiştir.

Veri seti 10 değişkenden oluşmaktadır:
* **step:** adım
* **type:** çevrimiçi işlem türü
* **amount:** işlem miktarı
* **nameOrig:** işlemi başlatan müşteri
* **oldbalanceOrg:** işlem öncesi bakiye
* **newbalanceOrig:** işlem sonrası bakiye
* **nameDest:** işlemin alıcısı
* **oldbalanceDest:** alıcının işlem öncesi başlangıç bakiyesi
* **newbalanceDest:** alıcının işlem sonrası yeni bakiyesi
* **isFraud:** dolandırıcılık işlemi

### Gerekli Kütüphaneleri Import Etme

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

### Veri Setini Yükleme

In [3]:
dataset = pd.read_csv('/content/contentdrive/MyDrive/Colab Notebooks/PS_20174392719_1491204439457_log.csv')

`head()` fonksiyonunu kullanarak veri setimizdeki ilk beş örneği gözlemleyebiliriz.

In [4]:
dataset.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


`dataset.shape` metoduyla veri setinin boyutlarını döndürelim.

In [5]:
dataset.shape

(6362620, 11)

### Boş Değerleri Kontrol Etme

**Checking for Null Values** (Boş Değerleri Kontrol Etme), bir veri setinde eksik veya boş değerlerin olup olmadığını kontrol etme işlemidir. Veri setlerinde eksik veriler, genellikle "NaN" (Not a Number) veya `None` olarak gösterilir ve bu değerlerin tespit edilip doğru şekilde ele alınması önemlidir.

In [6]:
dataset.isnull().sum()

,0
step,0
type,0
amount,0
nameOrig,0
oldbalanceOrg,0
newbalanceOrig,0
nameDest,0
oldbalanceDest,0
newbalanceDest,0
isFraud,0


Gereksiz sütunları çıkaralım.

In [7]:
dataset = dataset.drop(['nameOrig', 'nameDest'], axis=1)

Kategorik sütunları sayısal verilere dönüştürün.

In [8]:
dataset = pd.get_dummies(dataset, drop_first=True)

### Exploratory Data Analysis (EDA)

Bir veri setinin temel özelliklerini anlamak, örüntüleri keşfetmek, olası sorunları belirlemek ve veri hakkında öngörüler elde etmek amacıyla yapılan ilk analiz aşamasıdır. Makine öğrenmesi ve istatistiksel analizlerde başarılı bir model geliştirmek için oldukça önemlidir.

Bu kod parçası, veri setindeki sütunlar arasındaki **korelasyon** değerini hesaplayıp, özellikle `isFraud` sütunuyla diğer değişkenler arasındaki korelasyonları sıralar.

Korelasyon, iki değişken arasındaki doğrusal ilişkiyi ölçer ve -1 ile +1 arasında bir değer alır.

* **+1:** Güçlü pozitif ilişki (bir değişken artarken diğeri de artar).
* **-1:** Güçlü negatif ilişki (bir değişken artarken diğeri azalır).
* **0:** Hiçbir doğrusal ilişki yoktur.

In [9]:
correlation = dataset.corr()

correlation["isFraud"].sort_values(ascending=False)

,isFraud
isFraud,1.000000
amount,0.076688
type_TRANSFER,0.053869
isFlaggedFraud,0.044109
step,0.031578
type_CASH_OUT,0.011256
oldbalanceOrg,0.010154
newbalanceDest,0.000535
type_DEBIT,-0.002911
oldbalanceDest,-0.005885


sklearn kütüphanesinden train_test_split fonksiyonunu import edelim. Bu, veri setlerini 'train' ve 'test' alt kümelerine bölmek için kullanılır.

In [10]:
from sklearn.model_selection import train_test_split

Veri setimizi train ve test setlerine bölelim.
test_size=0.3: Test setinin boyutunu belirler. 0.3, veri setinin %30'unun test seti için kullanılacağı anlamına gelir. Dolayısıyla, %70'i train seti olacaktır.

In [11]:
train, test = train_test_split(dataset, test_size=0.3)


Tahmin etmek istediğimiz 'target'ı tanımlamalıyız.
Dolayısıyla target değişkeni 'isFraud' dur ve geri kalan sütunlar 'features' tur. Target olarak x değişkeni ve features olarak y değişkenini atayalım.

In [12]:
x_train = train.drop('isFraud', axis=1)
y_train = train.loc[:, 'isFraud']

Aynı şeyi test veri seti için de yapmamız gerekir.

In [13]:
x_test = test.drop('isFraud', axis=1)
y_test = test.loc[:, 'isFraud']

### LinearSVC Modelinin Kullanımı

LinearSVC modelini oluşturun ve eğitin

In [14]:
model = LinearSVC()
model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

### Değerlendirme

In [15]:
predictions = model.predict(x_test)
predictions

array([0, 0, 0, ..., 0, 0, 0])

Modelimizin doğruluğunu kontrol edelim.

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[1905725,     569],
       [    639,    1853]])

Modelin performansını değerlendirin

In [17]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906294
           1       0.77      0.74      0.75      2492

    accuracy                           1.00   1908786
   macro avg       0.88      0.87      0.88   1908786
weighted avg       1.00      1.00      1.00   1908786



## Denetimsiz Öğrenme

### Gerekli Kütüphaneleri Import Etme

In [27]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import adjusted_rand_score, homogeneity_score, completeness_score, v_measure_score
from sklearn.metrics import davies_bouldin_score
import warnings
warnings.filterwarnings("ignore")

### Veri Setini Yükleme

In [28]:
dataset = pd.read_csv('/content/contentdrive/MyDrive/Colab Notebooks/PS_20174392719_1491204439457_log.csv', nrows=30000)

### Hedef Değişkeninin Çıkarılması

Veri setinizdeki `isFraud` sütunu, denetimli öğrenme modeli için kullanılır. Denetimsiz öğrenme modellerinde, modelin etiket (hedef) değişkeni olmadan verilerdeki gizli yapıları, kalıpları veya kümeleri bulması amaçlanır. Bu yüzden bu değişkeni modelden çıkarıyoruz.

In [29]:
dataset = dataset.drop('isFraud', axis=1)

In [30]:
dataset = pd.get_dummies(dataset, drop_first=True)

### Küme Sayısı Seçimi

Küme sayısını 3'e ayarlayacağız.

In [31]:
model = KMeans(n_clusters=3)

### KMeans Cluster Modelinin Kullanımı

In [32]:
model.fit(dataset)

KMeans(n_clusters=3)

### Şimdi Tahminler Yapmaya Hazırız

Kümeleme durumunda (predictions) tahminlerin, örneğin ait olduğu grup sayısını gösterdiğini unutmayın.

Bu satır, önceden eğitilmiş KMeans modelini (model) kullanarak  'dataset' teki her veri noktası için küme etiketlerini tahmin eder.

In [33]:
labels = model.predict(dataset)

Küme sayısını 3'e ayarladığımızda üç etiket (0, 1 ve 2) elde ettiğimizi unutmayın.

Her grupta kaç örneğimiz olduğunu kontrol edelim.

In [34]:
np.unique(labels, return_counts=True)

(array([0, 1, 2], dtype=int32), array([26254,  1537,  2209]))

### Değerlendirme

Kümeleri değerlendirmek için Adjusted Rand Index kullanalım.

In [36]:
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(dataset, model.labels_)
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.8002778955314798


* **Yüksek Skor (0.8):** Bu, kümeler arasındaki ayrımın oldukça iyi olduğunu ve her bir örneğin doğru kümeye yerleştirildiğini gösterir. Örnekler, kendi kümelerinde iyi gruplandırılmış ve diğer kümelerden belirgin şekilde ayrılmıştır.

In [37]:
db_index = davies_bouldin_score(dataset, model.labels_)
print(f"Davies-Bouldin Index: {db_index}")

Davies-Bouldin Index: 0.6036441793442805


**DBI** değerinin 0.603 olması oldukça düşüktür ve kümelerin iyi ayrıldığını gösterir ve içsel olarak tutarlı olduğunu gösterir.

* **0'a yaklaştıkça**, kümeler arası mesafeler artarken, aynı küme içindeki öğeler birbirine daha yakındır. Bu da daha başarılı bir kümeleme demektir.
* **Daha yüksek değerler** (1 ve üstü), kümeler arası farkın düşük olduğunu ve kümelerin birbirine çok yakın olduğunu gösterir, bu da kötü bir kümeleme anlamına gelir.


In [38]:
print(f"Inertia: {model.inertia_}")

Inertia: 2.5654666442324134e+17
